## KNN으로 digit.png 학습

In [22]:
import sys
import numpy as np
import cv2

oldx, oldy = -1, -1

def on_mouse(event, x, y, flags, _):
    global oldx, oldy
    
    if event == cv2.EVENT_LBUTTONDOWN:
        oldx, oldy = x, y
    
    elif event == cv2.EVENT_LBUTTONUP:
        oldx, oldy = -1, -1
        
    elif event == cv2.EVENT_MOUSEMOVE:
        if flags & cv2.EVENT_FLAG_LBUTTON:
            cv2.line(img, (oldx, oldy), (x, y), (255, 255, 255), 20, cv2.LINE_AA)
            oldx, oldy = x, y
            cv2.imshow('img', img)
            
digits = cv2.imread('./images/digits.png', cv2.IMREAD_GRAYSCALE)

if digits is None:
    print('Image load failed')
    sys.exit()
    
h, w = digits.shape[:2]

cells = [np.hsplit(row, w//20) for row in np.vsplit(digits, h//20)] # hsplit열방향 분할 vsplit행방향분할
cells = np.array(cells)

train_images = cells.reshape(-1,400).astype(np.float32)
train_labels = np.repeat(np.arange(10), len(train_images)/10)

# training KNN
knn = cv2.ml.KNearest_create()
knn.train(train_images, cv2.ml.ROW_SAMPLE, train_labels)

img = np.zeros((400, 400), np.uint8)

cv2.imshow('img', img)
cv2.setMouseCallback('img', on_mouse)

while True:
    key = cv2.waitKey()
    
    if key == 27:
        break
    elif key == ord(' '):
        img_resize = cv2.resize(img, (20,20), interpolation=cv2.INTER_AREA)
        img_flatten = img_resize.reshape(-1, 400).astype(np.float32)
        
        _res, res, _, _  = knn.findNearest(img_flatten,3)
        print(int(res[0, 0]))
        
        img.fill(0)
        
        cv2.imshow('img', img)
        
cv2.destroyAllWindows()

1
1
0
1
2
3
1
4
5
4
3
3
9
6
1
2
7
8
7
3
1
1
1
3
9
5
9
9
1
1
1
4
1
1
1
1
1
2
2
5
4
5
6
1
7
1
7
4
6
8
9


In [19]:
cv2.imshow('train',img_flatten)
cv2.waitKey()
cv2.destroyAllWindows()

In [14]:
print(train_labels)

[0 0 0 ... 9 9 9]


In [20]:
img_flatten.shape

(1, 400)

In [23]:
cells

array([[[[ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0]],

        [[ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0]],

        [[ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0]],

        ...,

        [[ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
    

In [6]:
import numpy as np
import cv2

cap = cv2.VideoCapture('./images/vtest.avi')
ret, iback = cap.read()
# back = uint8배경, fback = float32배경 
iback = cv2.cvtColor(iback, cv2.COLOR_BGR2GRAY)
iback = cv2.GaussianBlur(iback, (0, 0), 1.0)
fback = iback.astype(np.float32)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (0,0), 1.0)
    
    cv2.accumulateWeighted(gray, fback, 0.01)
    back = fback.astype(np.uint8)
    diff = cv2.absdiff(gray, back)
    _, diff = cv2.threshold(diff, 30,255, cv2.THRESH_BINARY)
    cnt, _, stats, _ = cv2.connectedComponentsWithStats(diff)
    for i in range(1, cnt):
        x, y, w, h, s = stats[i]
        if s < 100: continue
        cv2.rectangle(frame, (x,y,w,h), (0,0,255), 2)
        
    cv2.imshow('frame', frame)
    cv2.imshow('diff', diff)
    cv2.imshow('back', back)
    cv2.imshow('iback', iback)
    
    if cv2.waitKey(30) == 27: break
        
cap.release()
cv2.destroyAllWindows()

In [5]:
# Gaussian Mixture model
import numpy as np
import cv2
cap = cv2.VideoCapture('./images/vtest.avi')

bs = cv2.createBackgroundSubtractorMOG2()# 배경차분알고리즘객체

while True:
    ret, frame = cap.read()
    if not ret: break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    fgmask = bs.apply(gray)
    back = bs.getBackgroundImage()
    cnt, _, stats, _ = cv2.connectedComponentsWithStats(fgmask)
    for i in range(cnt):
        x, y, w, h, s = stats[i]
        if s < 80:continue
        cv2.rectangle(frame, (x, y, w, h), (0,0,255), 2)
    cv2.imshow('frame', frame)
    cv2.imshow('back', back)
    cv2.imshow('fgmask', fgmask)
    if cv2.waitKey(20)==27:break
cap.release()
cv2.destroyAllWindows()

In [7]:
# tracking
import sys
import numpy as np
import cv2
cap = cv2.VideoCapture('./images/vtest.avi')
ret, frame = cap.read()
x, y, w, h = cv2.selectROI('Select Window', frame, fromCenter=False, showCrosshair=True)
rc = (x, y, w, h)
roi = frame[y:y+h, x:x+w]
roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# histogram
channels = [0, 1]
ranges = [0, 180, 0, 256]
hist = cv2.calcHist([roi_hsv], channels, None, [90, 128], ranges)
while True:
    ret, frame = cap.read()
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    backproj = cv2.calcBackProject([frame_hsv], channels, hist, ranges, 1)
    # Mean Shift
    _, rc = cv2.meanShift(backproj, rc, (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1))
    # 추적 결과 화면 출력
    cv2.rectangle(frame, rc, (0, 0, 255), 2)
    cv2.imshow('frame', frame)
    if cv2.waitKey(60)==27: break
        
cap.release()
cv2.destroyAllWindows()